In [1]:
import numpy as np
import tensorflow as tf
import os, shutil
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential

from MusicRnnData import MusicRnnData

# input parameters
x_len = 50
y_len = 1
batch_size = 4096
# LSTM parameters
num_layers = 1
lstm_size = 8
hidden_size = 8
# training parameters
dropout_prob = 0.5
num_epochs = 100
verbose = True
display_interval = 500
moving_avg_length = 100

# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


## Load data

In [2]:
# filelist = ['a2002011001-e02.wav']
filelist = ['sine.wav']
music_data = MusicRnnData(filelist)
x_train, y_train = music_data.batch(x_len, y_len, batch_size)

# reshape to be (samples, timesteps, features)
x_train = np.reshape(x_train, (batch_size, x_len, 1))
y_train = np.reshape(y_train, (batch_size, y_len))
print(x_train.shape)
print(y_train.shape)

(4096, 50, 1)
(4096, 1)


## Build network

In [3]:
def build_model(x_len, lstm_size, dropout_prob):
    model = Sequential()
    model.add(LSTM(input_shape=(x_len, 1), output_dim=(lstm_size), return_sequences=True))
    model.add(Dropout(dropout_prob))
    model.add(LSTM(lstm_size, return_sequences=False))
    model.add(Dropout(dropout_prob))
    model.add(Dense(output_dim=1))
    model.add(Activation("linear"))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

model = build_model(x_len, lstm_size, dropout_prob)

/home/bricklayer/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(units=8, return_sequences=True, input_shape=(50, 1))`
  app.launch_new_instance()
/home/bricklayer/anaconda2/envs/tensorflow/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`


## Train model

In [4]:
model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size, verbose=2)

Epoch 1/100
1s - loss: 1.3450e-04
Epoch 2/100
0s - loss: 5.3323e-05
Epoch 3/100
0s - loss: 5.5453e-05
Epoch 4/100
0s - loss: 6.5212e-05
Epoch 5/100
0s - loss: 1.1675e-04
Epoch 6/100
0s - loss: 6.6829e-05
Epoch 7/100
0s - loss: 4.6900e-05
Epoch 8/100
0s - loss: 3.8233e-05
Epoch 9/100
0s - loss: 2.8635e-05
Epoch 10/100
0s - loss: 7.0346e-05
Epoch 11/100
0s - loss: 7.9006e-05
Epoch 12/100
0s - loss: 7.6673e-05
Epoch 13/100
0s - loss: 6.3901e-05
Epoch 14/100
0s - loss: 4.0761e-05
Epoch 15/100
0s - loss: 6.0455e-05
Epoch 16/100
0s - loss: 5.3393e-05
Epoch 17/100
0s - loss: 4.2557e-05
Epoch 18/100
0s - loss: 6.7529e-05
Epoch 19/100
0s - loss: 7.1953e-05
Epoch 20/100
0s - loss: 7.0459e-05
Epoch 21/100
0s - loss: 6.6952e-05
Epoch 22/100
0s - loss: 2.0174e-05
Epoch 23/100
0s - loss: 5.7420e-05
Epoch 24/100
0s - loss: 7.0705e-05
Epoch 25/100
0s - loss: 7.0500e-05
Epoch 26/100
0s - loss: 6.6173e-05
Epoch 27/100
0s - loss: 7.7664e-05
Epoch 28/100
0s - loss: 6.2733e-05
Epoch 29/100
0s - loss: 6.259

## Predict on sequence

## Plot output